·原始数据处理

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def sum_accum(df,N):     #求某一段区间内的累计和的函数
    sume = list()
    a=[0]
    for i in range(N):
        a = a + df.values[i]
        a.tolist()
        sume.extend(a)
    return sume

In [3]:
def data_processing(data):
    
    ######将读取的原始数据进行简单处理，以日期作为索引，并将"stock","close","volume","turnover","value"作为列名#########
    data = data.set_index('Trddt')
    data.columns = ["stock","close","volume","turnover","value"]
    data.index.name = "date"

    stock_sum = pd.DataFrame(columns = ['stock', 'close', 'turnover_sum', 'MV_ratio', 'change'])  ##建立一个空的dataframe

    ######对所有股票进行循环，并将其存入stock_sum中#######################
    for i in list(set(data["stock"])):
        stock_eg = data[data["stock"] == i]
        stock_eg = stock_eg.sort_index()                  #将dataframe按照索引从小到大排列
        turnover = stock_eg['turnover']
        close = stock_eg["close"]
        MV = stock_eg['value']
        turnover_sum = sum_accum(turnover,len(stock_eg))
        MV_ratio = sum_accum(turnover/MV,len(stock_eg))
        change = close.pct_change(periods=1, fill_method='pad')    #求每日涨跌函数
        stock_eg["turnover_sum"] = turnover_sum
        stock_eg['MV_ratio'] = MV_ratio
        stock_eg["change"] = change
        stock_eg = stock_eg[['stock', 'close', 'turnover_sum', 'MV_ratio', 'change']]
        stock_sum = stock_sum.append(stock_eg)
    
    stock_sum['change_sum'] = stock_sum['change'].groupby(stock_sum['stock']).cumsum()
    stock_sum.index.name = 'date'
    return stock_sum  

In [4]:
data = pd.read_table('D:/Documents/Go Abroad/实习/成交金额/原始数据/2017.txt',encoding='gbk',engine='python')

In [5]:
stock = data_processing(data)

In [6]:
stock.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017_2/2017.csv')

·成交金额市值比及涨跌幅

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数
    #data = data[['date','stock','close','turnover_sum','change']]
    #data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'turnover_sum', 'change', 'change_sum', 'turnover_N1', 'turnover_N2', 'MV_N', 'change_N'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        turn_o = starter['turnover_sum']
        change_s = starter['change_sum']
        MV_s = starter['MV_ratio']
        differ_t = turn_o.values[1]-turn_o.values[0]                  ###计算T前N1天的成交金额总量
        differ_t2 = turn_o.values[3]-turn_o.values[1]                 ###计算T后N2天的成交金额总量
        differ_m = MV_s.values[1]-MV_s.values[0]                      ###计算T前N1天的市值比总量
        differ_c = change_s.values[3]-change_s.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(6,'turnover_N1',[differ_t])
        stock_a.insert(7,'turnover_N2',[differ_t2])
        stock_a.insert(8,'MV_N',[differ_m])
        stock_a.insert(9,'change_N',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)

    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date','stock','close','turnover_N1','turnover_N2', 'MV_N', 'change_N']]
    
    stock_Tmax = stock_f.sort_values(by = ['MV_N'], ascending = False).head(K)     ###取出N天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change_N'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    

In [4]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017_2/2017.csv',encoding='gbk',engine='python')

In [5]:
#stock_m = max_stock(data)

stock_eg = data[data['stock'] == 1]

In [6]:
stock_eg

,date,stock,close,turnover_sum,MV_ratio,change,change_sum
0,2017-01-03,1,9.16,4.205952e+08,3.138264,NaN,NaN
1,2017-01-04,1,9.16,8.320986e+08,6.208690,0.000000,0.000000
2,2017-01-05,1,9.17,1.147868e+09,8.562231,0.001092,0.001092
3,2017-01-06,1,9.13,1.475045e+09,11.011474,-0.004362,-0.003270
4,2017-01-09,1,9.15,1.805039e+09,13.143227,0.002191,-0.001080
5,2017-01-10,1,9.15,2.025614e+09,14.568135,0.000000,-0.001080
6,2017-01-11,1,9.14,2.303168e+09,16.363080,-0.001093,-0.002173
7,2017-01-12,1,9.15,2.695037e+09,18.894543,0.001094,-0.001079
8,2017-01-13,1,9.16,3.092639e+09,21.460233,0.001093,0.000014
9,2017-01-16,1,9.14,3.715665e+09,25.489362,-0.002183,-0.002169


In [7]:
stock_bestinf = best_perf(data, stock_eg, '2017-04-07', 60, 30, 100)

In [8]:
stock_bestinf

,date,stock,close,turnover_N1,turnover_N2,MV_N,change_N
2188,2017-04-07,11,16.91,8.801985e+09,7.512055e+09,NaN,0.003842
4658,2017-04-07,22,21.98,7.198629e+09,9.839750e+09,NaN,0.004047
12421,2017-04-07,62,25.92,4.108068e+09,6.076053e+09,NaN,0.010946
12657,2017-04-07,63,17.26,4.043515e+10,1.743591e+10,NaN,0.003548
14118,2017-04-07,69,7.51,9.161213e+09,1.140383e+10,NaN,0.004422
15764,2017-04-07,88,8.75,5.833334e+09,1.558552e+10,NaN,0.003681
20494,2017-04-07,158,12.44,4.103636e+09,2.491235e+10,NaN,0.010029


·统计各交易日情况

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数
    #data = data[['date','stock','close','turnover_sum','change']]
    #data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'turnover_sum', 'change', 'change_sum', 'turnover_N1', 'turnover_N2', 'MV_N', 'change_N'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        turn_o = starter['turnover_sum']
        change_s = starter['change_sum']
        MV_s = starter['MV_ratio']
        differ_t = turn_o.values[1]-turn_o.values[0]                  ###计算T前N1天的成交金额总量
        differ_t2 = turn_o.values[3]-turn_o.values[1]                 ###计算T后N2天的成交金额总量
        differ_m = MV_s.values[1]-MV_s.values[0]                      ###计算T前N1天的市值比总量
        differ_c = change_s.values[3]-change_s.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(6,'turnover_N1',[differ_t])
        stock_a.insert(7,'turnover_N2',[differ_t2])
        stock_a.insert(8,'MV_N',[differ_m])
        stock_a.insert(9,'change_N',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)

    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date','stock','close','turnover_N1','turnover_N2', 'MV_N', 'change_N']]
    
    stock_Tmax = stock_f.sort_values(by = ['MV_N'], ascending = False).head(K)     ###取出N天成交金额市值比最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change_N'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    

In [4]:
def statisc(data, stock_eg, N1, N2, K):
    
    statis = pd.DataFrame(columns = ['date','stock','close','turnover_N1','turnover_N2','change_N'])

    for i in stock_eg['date'].values[N1+1:-N2+1]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N1, N2, K)
            statis = statis.append(stock_bestinf, ignore_index=True)
        except:
            continue

    return statis

In [5]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2017_2/2017.csv',encoding='gbk',engine='python')

In [6]:
stock_m = 1
#stock_m = max_stock(data)
stock_eg = data[data['stock'] == stock_m]

In [7]:
statis = statisc(data, stock_eg, 60, 30, 100)
statis

,MV_N,change_N,close,date,stock,turnover_N1,turnover_N2
0,8760.825119,0.007910,53.83,2017-04-07,2837,9.639468e+09,4.093583e+09
1,7511.008153,0.007591,13.07,2017-04-07,600908,1.795445e+10,1.666955e+10
2,10837.462885,0.007233,130.56,2017-04-07,300571,1.232685e+10,4.054402e+09
3,6278.492456,0.009371,83.55,2017-04-07,300577,8.000032e+09,2.499408e+09
4,8834.755035,0.004539,51.73,2017-04-10,2837,9.719104e+09,4.317166e+09
5,7586.617636,0.006339,13.38,2017-04-10,600908,1.814617e+10,1.721181e+10
6,10933.624520,0.005464,122.00,2017-04-10,300571,1.245366e+10,4.057310e+09
7,6401.484376,0.009609,82.26,2017-04-10,300577,8.172051e+09,2.460254e+09
8,8922.982771,0.005900,52.24,2017-04-11,2837,9.810443e+09,4.504885e+09
9,7754.339980,0.004631,14.50,2017-04-11,600908,1.857281e+10,1.757083e+10


In [8]:
statis.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017_2/2017-60-30.csv', index=False)

·出现上述情况的各股票次数

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import datetime
%matplotlib inline

In [4]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2017_2/2017-60-30.csv',encoding='gbk',engine='python')

In [5]:
data = data.sort_values(by = 'stock')
data = data.reset_index(drop=True)  

In [6]:
counter = data.groupby('stock').size()

In [7]:
stock_id = list(set(data['stock']))
stock_id.sort()

In [8]:
computa = pd.DataFrame({'stock': stock_id, 'count': list(counter)}, columns = ['stock', 'count'])
computa

,stock,count
0,615,10
1,877,10
2,2307,6
3,2346,9
4,2457,11
5,2497,11
6,2742,23
7,2809,12
8,2819,3
9,2830,40


In [9]:
computa.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2017_2/counter6030.csv', index = False)

·成交金额与涨跌幅正相关性较强股票

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import datetime
%matplotlib inline

In [10]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2017_2/counter6030.csv',encoding='gbk',engine='python')

In [11]:
data_fit = data[data['count'] >= 15]
data_fit = data_fit.sort_values(by = 'count', ascending = False)
data_fit = data_fit.reset_index(drop = True)
data_fit

,stock,count
0,300618,65
1,2830,40
2,300554,32
3,603938,29
4,300619,27
5,603286,26
6,603896,23
7,300666,23
8,2742,23
9,300609,22
